# Test autoFRK

## Install our python autoFRK

In [ ]:
print("=" * 50)
print("Python Environment Info")
print("=" * 50)
print(f"Python executable: {sys.executable}")
print(f"Python version: {sys.version.split()[0]}")
print()

print("=" * 50)
print("Package Locations")
print("=" * 50)
print(f"Torch location: {torch.__file__}")
print(f"NumPy location: {np.__file__}")
print()

print("=" * 50)
print("PyTorch Info")
print("=" * 50)
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA build: {torch.version.cuda}")
print()

print("=" * 50)
print("Test PyTorch Computation")
print("=" * 50)
x = torch.rand(3, 3)
print(f"Random tensor:\n{x}")
print(f"Sum: {x.sum().item():.4f}")

In [ ]:
# install autoFRK in development mode
import os
import sys
module_root = os.path.abspath(os.path.join(os.getcwd(), ".."))

!"{sys.executable}" -m pip uninstall -y autoFRK
!"{sys.executable}" -m pip install --upgrade pip build setuptools wheel matplotlib
!"{sys.executable}" -m build {module_root}
!"{sys.executable}" -m pip install -e "{module_root}"

## Import modules

In [ ]:
# import modules
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
from autoFRK import AutoFRK
from autoFRK.utils.utils import to_tensor

## Load data

In [ ]:
# load data
datasets_path = f'test datasets/matrixForTest'
data = pd.read_csv(os.path.join(datasets_path, 'matrixForTest_data.csv'))
locs = pd.read_csv(os.path.join(datasets_path, 'matrixForTest_locs.csv'))

In [ ]:
data

In [ ]:
locs

## Convert data to tensor

In [ ]:
# convert to tensor
data = to_tensor(data.to_numpy())
locs = to_tensor(locs.to_numpy())

## Test on known locations

In [ ]:
model = AutoFRK()
model.forward(
    data=data,
    loc=locs
)

In [ ]:
pred = model.predict()

In [ ]:
import torch.nn.functional as F

F.mse_loss(pred['pred.value'], data).item()

```r
> options(digits = 15)
> print(mse)
[1] 11.8539587159961 # R mse result
```

In [ ]:
for i in range(data.shape[1]):
    y_pred = pred['pred.value'][:, i]
    y_true = data[:, i]

    tmp = F.mse_loss(y_pred, y_true)
    print(tmp)

    plt.figure(figsize=(8,5))
    plt.plot(y_true, label='True Value', marker='o')
    plt.plot(y_pred, label='Predicted Value', marker='x')
    plt.xlabel('Sample Index')
    plt.ylabel('Value')
    plt.title(f'Prediction vs True Value at {i} Step with error {round(tmp.item(), 2)}')
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
for i in range(data.shape[1]):
    y_pred = pred['pred.value'][:, i]
    y_true = data[:, i]

    tmp = F.mse_loss(y_pred, y_true)
    print(tmp.item())

### Compare with r result

```r
> temp # r residual result
 [1]  7.9766306122139898 12.0116817669945561  8.0081251964859330  3.5373058520311229 13.8528751726772583
 [6]  1.7858863206045086  9.8859536589469261 30.0424029115677840 27.0729660181976612  0.2159766252862333
[11]  0.0752655442993194  5.8887993596240475 12.5987205947255649 11.4932438517303979 37.7533887912337889
[16]  1.7491903038108714  4.2473651696074111  6.6476433759056297  4.3364578111025098  5.5149082237930296
[21] 23.4936795233438822  5.3612826806645080 10.3639904061309522 22.0650623239508086  3.2049127751009370
[26]  5.1869105672897371 30.8554320276935812  1.3647053212213505  9.0990348107428840 39.9289638829070839
```

經過確認，R 與 Python 的結果是大致一致的。
全時間點與各時間點的 MSE ，與 R 的結果皆相差不大，精準度介於小數點後6位元至10位元。

## Test on unknown locations
### Split data into train and test

In [ ]:
## split data into train and test
train_data = data[:70, :]
test_data = data[70:, :]
train_locs = locs[:70, :]
test_locs = locs[70:, :]

## training on train data
model = AutoFRK()
model.forward(
    data=train_data,
    loc=train_locs
)

## predict on test data
pred = model.predict(
    newloc = test_locs
)
F.mse_loss(pred['pred.value'], test_data).item()
for i in range(test_data.shape[1]):
    y_pred = pred['pred.value'][:, i]
    y_true = test_data[:, i]

    tmp = F.mse_loss(y_pred, y_true)
    print(tmp.item())

In [ ]:
def p(obj):
    """
    遞迴漂亮印出 dict / list / tensor。
    tensor 會換行縮排對齊，方便查看。
    """
    def pretty_tensor(tensor: torch.Tensor, indent: int = 6) -> str:
        lines = str(tensor).split("\n")
        if len(lines) == 1:
            return f"{lines[0]}"
        ind = " " * indent
        return "\n" + "\n".join(ind + line for line in lines) + "\n" + " " * (indent - 2)

    def _p(obj, indent=0):
        space = " " * indent
        if isinstance(obj, dict):
            print(space + "{")
            for k, v in obj.items():
                print(f"{space}  {repr(k)}: ", end="")
                _p(v, indent + 4)
            print(space + "}")
        elif isinstance(obj, list) or isinstance(obj, tuple):
            print(space + "[")
            for v in obj:
                _p(v, indent + 4)
            print(space + "]")
        elif isinstance(obj, torch.Tensor):
            print(pretty_tensor(obj, indent + 4))
        else:
            print(repr(obj))

    _p(obj)

In [ ]:
p(model.obj)

### Compare with r result